In [1]:
# ------------------------------------------------------------------------------------
#
#           Code to read in _Network file and add reservoir information 
#
# ------------------------------------------------------------------------------------

In [20]:
import sys
import csv
import pandas as pd
import os.path
import numpy as np

In [61]:
# ------------------------------------------------------------------------------------
#        read in reservoir information, write all nodes with a reservoir
# ------------------------------------------------------------------------------------

reservoir_file = '/raid3/rniemeyr/RBM/two_layer_model/RBM_Yixin/RIPS/model_run/source/VIC_RBM/tools/cfg/reservoir_to_model_total.csv'
#reservoir_file = './cfg/reservoir_to_model.csv'
# reservoir_file = 'reservoir_to_model.csv'
# reservoir_file = sys.argv[2]

reservoir_nodes = pd.read_csv(reservoir_file)

# -------- sequence of numbers -----------
reservoir_tot_nodes = []
reservoir_tot_nodes_index = []
for i in range(0, len(reservoir_nodes['start_node']) ):
    x = range(int(reservoir_nodes['start_node'][i]), int(reservoir_nodes['end_node'][i])+1 )
    reservoir_tot_nodes.extend(x)
    # x = repeat(i+1, len(x) )  # want index to start with 1, not 0
    x = np.repeat(i+1, len(x))
    reservoir_tot_nodes_index.extend(x)

reservoir_tot_nodes2 = pd.DataFrame(reservoir_tot_nodes)

print(reservoir_tot_nodes_index)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 12, 12, 12, 14, 14, 15, 15, 15, 16, 16, 17, 17, 17, 17, 18, 18, 19, 19, 20, 21, 21, 21, 21, 21, 21, 22, 23, 24, 24, 24, 24, 24, 25]


In [62]:
# ------------------------------------------------------------------------------------
#        read in reservoir information, write all nodes with a reservoir
# ------------------------------------------------------------------------------------

network_file = '../RBM_processing/Tennessee_8th.Mohseni_v1/Tennessee_8th_Network'
#network_file = '../RBM_processing/Holston_8th.Mohseni_v1/Holston_8th_Network'
# network_file = "Holston_8th_Network"
# network_file = sys.argv[1]
with open(network_file) as f:
    network =  f.read().splitlines()

# -------- get dimensions of river network ----------    
str1 = network[4]
dim = [int(s) for s in str1.split() if s.isdigit()]  # nreach, flow_cells, heat_cells, source

# ------------- reform the network file ------------
network2 = network[0:3]
network2.append(reservoir_file) # instert path to reservoir info
network2.append(network[3])   # add start and end date
x = network[4] +  str(len(reservoir_nodes)).rjust(6) + '    TRUE'  # a boolean for if reservoirs are simulated with two-layer models
network2.append(x)

In [63]:
print(network2)

['RBM parameter preparation control file template', '/raid2/ymao/VIC_RBM_east_RIPS/RIPS/model_run/output/RVICandVIC_for_RBM/Tennessee.hist_1949_2010.calibrated_1961_1970.Mohs_v1.Leop_v1.reservoir/Tennessee.DA_flow', '/raid2/ymao/VIC_RBM_east_RIPS/RIPS/model_run/output/RVICandVIC_for_RBM/Tennessee.hist_1949_2010.calibrated_1961_1970.Mohs_v1.Leop_v1.reservoir/Tennessee.DA_energy', '/raid3/rniemeyr/RBM/two_layer_model/RBM_Yixin/RIPS/model_run/source/VIC_RBM/tools/cfg/reservoir_to_model_total.csv', '            19490101               20101231', '       299       665       964                FALSE    25    TRUE']


In [64]:
# ------------------------------------------------------------------------------------
#        loop to get the nodes each file
# ------------------------------------------------------------------------------------

for i in range(5, len(network)):
        x = network[i]
        if x[0:4] == 'Node':
                x2 = [int(s) for s in x.split() if s.isdigit()]
                
                # if node in NETWORK file has a reservoir 
                if x2[0] in reservoir_tot_nodes:
                       
                        nodesx = int(np.where(reservoir_tot_nodes2 == x2[0])[0])
                        nodesx2 = reservoir_tot_nodes_index[nodesx]
                        x4 = str(nodesx2)
                        #x4 = x4.rjust(5-len(x4))
                        x3 = network[i] + x4.rjust(6)  + '  TRUE '
                        #x3 = network[i] + '   ' +  x4 + '  TRUE '
                        network2.append(x3)
                        
                # if node in NETWORK file does not have reservoir
                else:
                        x3 = network[i] + '  ' +  '   0'  + '  FALSE'
                        network2.append(x3)
        else:
                network2.append(x)

In [65]:
# ------------- write the file -------- 

network_file = network_file + "_2"  # REMOVE once you have a network file you like (i.e. overwrite network file)
save_path = '/raid3/rniemeyr/RBM/two_layer_model/RBM_Yixin/RIPS/model_run/source/VIC_RBM/RBM_processing/Tennessee_8th.Mohseni_v1/'
network_file2 = save_path + 'Tennessee_8th_Network_2'
# completeName = os.path.join(save_path, network_file)         

with open(network_file2, "w") as outfile:
    for counter in range(0, len(network2)):
        outfile.write(network2[counter] + '\n')


In [8]:
print(x4)

   1


In [17]:
xi = 

In [18]:
len(x4)

4

6